# Dados em painel (multi-series)

Em muitas aplicações, não temos acesso a uma única série temporal, mas sim a um conjunto de séries temporais relacionadas. Isso é comum em cenários como vendas de produtos em diferentes lojas, consumo de energia em diferentes regiões, etc. Esses dados são chamados de dados em painel.

Uma ideia poderosa é aproveitar a similaridade entre as séries para melhorar as previsões. Chamamos de **modelos globais** os modelos capazes de aprender padrões comuns entre as séries, ao contrário dos **modelos locais** que aprendem apenas com uma única série.

A maioria dos modelos clássicos de séries temporais são locais. Modelos globais são, em geral, baseados em modelos tabulares de ML ou deep learning. Segundo competições de séries temporais, como a M5, em forecasts de painel os modelos globais são os que apresentam melhor desempenho [@makridakis2022m5].


## Acessando os dados

Aqui, vamos usar o dataset sintético que vimos antes, mas agora teremos acesso às várias séries temporais que compõe o total.

Esse dataset é feito para simular um caso de varejo, onde temos vendas diárias de vários produtos:


In [ ]:
# | echo: false
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# | code-fold: true
import pandas as pd
import matplotlib.pyplot as plt

from sktime.utils.plotting import plot_series

In [ ]:
from tsbook.datasets.retail import SyntheticRetail
dataset = SyntheticRetail("panel")
y_train, X_train, y_test, X_test = dataset.load(
    "y_train", "X_train", "y_test", "X_test"
)

Note que, para dados em painel, os dataframes possuem mais um nível de índice, que identifica a série temporal a que cada observação pertence:


In [ ]:
display(X_train)

Podemos visualizar algumas séries. Vemos que há mais zeros nesse dataset, em comparação
ao que usamos antes.


In [ ]:
from sktime.utils.plotting import plot_series

fig, ax = plt.subplots(figsize=(10, 4))
y_train.unstack(level=0).droplevel(0, axis=1).iloc[:, [0,10]].plot(ax=ax, alpha=0.7)
plt.show()

### Pandas e multi-índices

Para trabalhar com essas estruturas de dados, é importante revisar algumas operações do pandas.


In [ ]:
y_train.index.get_level_values(-1)

As seguintes operações são bem úteis para trabalhar com multi-índices:


In [ ]:
y_train.index

Acessar valores únicos no primeiro nivel (nível 0, mais à esquerda):


In [ ]:
y_train.index.get_level_values(0).unique()

Selecionar uma série específica (nível 0 igual a 0):


In [ ]:
y_train.loc[0]

Aqui, podemos usar `pd.IndexSlice` para selecionar várias séries ao mesmo tempo.
Note que pd.IndexSlice é passado diretamente para `.loc`:


In [ ]:
y_train.loc[pd.IndexSlice[[0,2], :]]

Agora, para selecionar o horizonte de forecasting, temos que chamar `unique`:


In [ ]:
fh = y_test.index.get_level_values(1).unique()

fh

## Upcasting automático

Nem todos modelos suportam nativamente dados em painel. Por exemplo, exponential smoothing.
Aqui, temos uma boa notícia: sem linhas extras necessárias. O sktime faz *upcasting* automático para dados em painel ao usar estimadores do `sktime`.


In [ ]:
from sktime.forecasting.naive import NaiveForecaster


naive_forecaster = NaiveForecaster(strategy="last", window_length=1)
naive_forecaster.fit(y_train)
y_pred_naive = naive_forecaster.predict(fh=fh)

y_pred_naive

Internamente, o `sktime` cria um clone do estimador para cada série nos dados em painel.
Em seguida, cada clone é treinado com a série correspondente. Isso é feito de
forma transparente para usuário, mas sem exigir esforço.

O atributo `forecasters_` armazena um DataFrame com os estimatores de cada série.


In [ ]:
naive_forecaster.forecasters_.head()

É dificil explicar o quanto isso é extremamente útil para código limpo e prototipagem rápida.
Foi um dos motivos que me levaram a usar o `sktime`.


## Métricas

Agora que temos várias séries, precisamos explicar como calcular métricas de avaliação.
O sktime oferece duas opções para isso, como argumentos na criação da métrica:

* `multilevel="uniform_average_time"` para calcular a média das séries temporais no painel.
* `multilevel="raw_values"` para obter o erro por série.


In [ ]:
from sktime.performance_metrics.forecasting import MeanSquaredScaledError

metric = MeanSquaredScaledError(multilevel="uniform_average_time")

In [ ]:
metric(y_true=y_test, y_pred=y_pred_naive, y_train=y_train)

Na prática, as métricas que a sua aplicação exige podem ser diferentes. Por exemplo,
as séries temporais podem ter diferentes importâncias, e você pode querer ponderar
as métricas de acordo. 

Para isso, é possível criar uma métrica customizada no sktime, mas não entraremos
nesse mérito aqui.

## Modelos globais de Machine Learning

Quando vimos como usar modelos de Machine Learning para forecasting, já mencionamos
como é necessário traduzir o problema de séries temporais para um problema de regressão tradicional.

No caso de dados em painel, também podemos usar essa abordagem, mas agora aproveitando
todas as séries temporais para treinar um único modelo global.
 
![](img/global_reduction.png)

Abaixo, vamos comparar um LightGBM global com um local, e ver como o global
aproveita melhor os dados.


In [ ]:
from tsbook.forecasting.reduction import ReductionForecaster
from lightgbm import LGBMRegressor

global_forecaster1 = ReductionForecaster(
    LGBMRegressor(n_estimators=100),
    window_length=30,
)

global_forecaster1.fit(y_train, X_train)

In [ ]:
y_pred_global1 = global_forecaster1.predict(fh=fh, X=X_test)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
y_train.loc[10, "sales"].plot(ax=ax, label="Treino")
y_test.loc[10, "sales"].plot(ax=ax, label="Teste")
y_pred_global1.loc[10, "sales"].plot(ax=ax, label="Global 1")
plt.legend()
plt.show()

Para forçar que um modelo global funcione como um modelo local, podemos usar `ForecastByLevel`, que cria um modelo separado para cada série temporal, mesmo quando o estimador suporta dados em painel.


In [ ]:
from sktime.forecasting.compose import ForecastByLevel

local_forecaster = ForecastByLevel(global_forecaster1, groupby="local")

local_forecaster.fit(y_train, X=X_train)

In [ ]:
y_pred_local = local_forecaster.predict(fh=fh, X=X_test)

err_global = metric(y_true=y_test, y_pred=y_pred_global1, y_train=y_train)
err_local = metric(y_true=y_test, y_pred=y_pred_local, y_train=y_train)

pd.DataFrame(
    {
        "Global": [err_global],
        "Local": [err_local],
    },
    index=["MSE"],
)

## Preprocessamento e engenharia de features

Sabemos como preprocessar séries temporais univariadas para melhorar o desempenho dos modelos de ML. Aplicamos da mesma maneira que fizemos anteriormente o `Differencer`, com objetivo de remover tendências.


In [ ]:
from sktime.transformations.series.difference import Differencer
from sktime.transformations.series.boxcox import LogTransformer

global_forecaster2 = Differencer() * global_forecaster1
global_forecaster2.fit(y_train, X_train)

In [ ]:
y_pred_global2 = global_forecaster2.predict(fh=fh, X=X_test)
metric(y_true=y_test, y_pred=y_pred_global3, y_train=y_train)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
y_train.loc[0].plot(ax=ax, label="Treino")
y_test.loc[0].plot(ax=ax, label="Teste")
y_pred_global3.loc[0].plot(ax=ax, label="Global 4")
fig.show()

### Pipelines exógenos também para dados em painel!


In [ ]:
from sktime.transformations.series.fourier import FourierFeatures

fourier_features = FourierFeatures(sp_list=[365.25, 365.25/12], fourier_terms_list=[1, 1], freq="D")

global_forecaster4 = fourier_features ** global_forecaster3
global_forecaster4.fit(y_train, X_train)

In [ ]:
y_pred_global4 = global_forecaster4.predict(fh=fh, X=X_test)
metric(y_true=y_test, y_pred=y_pred_global4, y_train=y_train)

In [ ]:
metric(y_true=y_test, y_pred=y_pred_global4, y_train=y_train)